In [1]:
import scanpy as sc
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
#R interface
%load_ext rpy2.ipython

import rpy2.robjects as ro

from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [3]:
%%R
library('Seurat')
library(Matrix)
library(qs)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject

R[write to console]: qs 0.25.5



In [4]:
dir_data='/net/bmc-lab6/data/lab/kellis/users/khrovati/data/datasets/'

## human_brain_organoid_DS

In [14]:
dir_ds=dir_data+'human_brain_organoid_DS/'
fn='all_combined_cycling.qs'

In [7]:
%%R -i dir_ds -i fn -o obs -o var -o shape -o count_data_i -o count_data_j -o count_data_x
obj<-qread(paste0(dir_ds,fn))
shape<-dim(obj)
counts<-GetAssayData(object = obj, slot = "counts")
counts<-as(counts, "TsparseMatrix")
# Exporting like vectors is fastest (not exporting like df, etc)
# Matrix starts indexing at 0
count_data_i<-counts@i
count_data_j<-counts@j
count_data_x<-counts@x
obs<-obj@meta.data
var<-obj[['RNA']][[]]

In [8]:
%R dim(obj)

array([15706, 49320], dtype=int32)

In [9]:
counts=csr_matrix((count_data_x,(count_data_j,count_data_i)), shape=shape[::-1])

In [10]:
adata=sc.AnnData(counts,obs=obs,var=var)

/tmp/ipykernel_313046/1443015886.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata=sc.AnnData(counts,obs=obs,var=var)


In [11]:
adata

AnnData object with n_obs × n_vars = 49320 × 15706
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'RNA_snn_res.0.4', 'RNA_snn_res.0.8', 'RNA_snn_res.1.2', 'RNA_snn_res.1.6', 'RNA_snn_res.2', 'RNA_snn_res.2.4', 'RNA_snn_res.2.8', 'RNA_snn_res.3.2', 'RNA_snn_res.3.6', 'RNA_snn_res.4', 'seurat_clusters', 'umap_dbscan', 'FullLineage', 'LineComp', 'anno', 'Sample', 'Genotype', 'Time', 'anno_group', 'cycling_group', 'group_comb'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'

In [18]:
for col in adata.obs.columns:
    print('****\n',col)
    if adata.obs[col].nunique()<30:
        print(sorted(adata.obs[col].unique()))

****
 orig.ident
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
****
 nCount_RNA
****
 nFeature_RNA
****
 percent.mt
****
 RNA_snn_res.0.4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
****
 RNA_snn_res.0.8
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
****
 RNA_snn_res.1.2
****
 RNA_snn_res.1.6
****
 RNA_snn_res.2
****
 RNA_snn_res.2.4
****
 RNA_snn_res.2.8
****
 RNA_snn_res.3.2
****
 RNA_snn_res.3.6
****
 RNA_snn_res.4
****
 seurat_clusters
****
 umap_dbscan
[0, 1, 2, 3, 4]
****
 FullLineage
['IPs and early cortical neurons', 'RGCs 3', 'RGCs early 1', 'RGCs early 2', 'choroid plexus/mesenchymal-like cells', 'cortical neurons 1', 'cortical neurons 2', 'cycling dorsal progenitors', 'cycling ventral progenitors', 'ectodermal/neuroectodermal-like cells', 'gliogenic/outer RGCs and astrocytes', 'midbrain/hindbrain cells', 'neuroepithelial-like cells', 'retina progenitors', 'stem cells 1', 'stem cells 2', 'stem cells 3', 'vent

In [19]:
pd.crosstab(adata.obs.FullLineage,adata.obs.LineComp)

LineComp,Cerebellar glutamatergic neurons 1,Cerebellar glutamatergic neurons 2,Cortical IPs,Cortical NPCs,Cortical neurons,LGE interneurons,MGE/CGE interneurons,Mid-hindbrain NPCs,Midbrain GABAergic interneurons,Purkinje cells,Ventral NPCs
FullLineage,,,,,,,,,,,
IPs and early cortical neurons,28,0,802,5,283,0,0,7,0,67,1
RGCs 3,0,0,87,2131,0,0,0,6,0,0,2
RGCs early 1,11,1,129,680,0,0,0,8092,0,6,16
RGCs early 2,26,3,58,83,0,0,0,2477,0,9,0
choroid plexus/mesenchymal-like cells,47,3,150,103,2,0,5,17777,2,59,6
cortical neurons 1,10,1,19,0,2265,1,2,0,1,98,0
cortical neurons 2,8,0,27,1,172,0,0,0,0,12,0
cycling dorsal progenitors,0,0,124,659,0,0,0,1567,0,0,97
cycling ventral progenitors,0,0,0,3,0,0,0,2,0,0,6


In [20]:
pd.crosstab(adata.obs.LineComp,adata.obs.group_comb)

group_comb,Mesenchymal_Cycling,Mesenchymal_Non-cycling,Neural_Cycling,Neural_Non-cycling
LineComp,,,,
Cerebellar glutamatergic neurons 1,0,11,0,4415
Cerebellar glutamatergic neurons 2,0,0,0,139
Cortical IPs,0,9,101,1565
Cortical NPCs,3,69,872,2888
Cortical neurons,0,0,0,2737
LGE interneurons,0,0,0,31
MGE/CGE interneurons,0,0,0,110
Mid-hindbrain NPCs,2459,11010,3357,18431
Midbrain GABAergic interneurons,0,0,0,36


In [17]:
adata.write(dir_ds+'.'.join(fn.split('.')[:-1])+'.h5ad')

## mouse_brain_devel_genus

In [21]:
dir_ds=dir_data+'mouse_brain_devel_genus/'
fn='filtered_celltype_subset_labelled.rds'

In [23]:
%%R -i dir_ds -i fn -o obs -o var -o shape -o count_data_i -o count_data_j -o count_data_x
obj<-readRDS(paste0(dir_ds,fn))
shape<-dim(obj)
counts<-GetAssayData(object = obj, slot = "counts")
counts<-as(counts, "TsparseMatrix")
# Exporting like vectors is fastest (not exporting like df, etc)
# Matrix starts indexing at 0
count_data_i<-counts@i
count_data_j<-counts@j
count_data_x<-counts@x
obs<-obj@meta.data
var<-obj[['RNA']][[]]

In [24]:
%R dim(obj)

array([20478, 23376], dtype=int32)

In [25]:
counts=csr_matrix((count_data_x,(count_data_j,count_data_i)), shape=shape[::-1])

In [26]:
adata=sc.AnnData(counts,obs=obs,var=var)

/tmp/ipykernel_313046/1443015886.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata=sc.AnnData(counts,obs=obs,var=var)


In [27]:
adata

AnnData object with n_obs × n_vars = 23376 × 20478
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Genotype', 'ID', 'Treatment', 'percent.mt', 'percent.ribo', 'log10GenesPerUMI', 'RNA_snn_res.1', 'seurat_clusters', 'layerI1', 'layerV_VI_51', 'layerV_VI_131', 'layerV_VI_31', 'layerV_VI_71', 'svz1_migrating_41', 'svz2_migrating_111', 'svz3_prolif1', 'rg11', 'rg21', 'rg31', 'gang1', 'int11', 'int21', 'stri_inh_11', 'stri_inh_21', 'thalamic1', 'choroid_plexus1', 'mic1', 'endo1', 'Endo1', 'Fib1', 'Per1', 'SMC1', 'Ependymal1', 'celltype', 'celltype_mouse_neocortical_clusters', 'Genotype_Treatment', 'dbscan_cl'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'

In [32]:
for col in adata.obs.columns:
    print('****\n',col)
    if adata.obs[col].nunique()<30:
        print(sorted(adata.obs[col].unique()))

****
 orig.ident
['DS_CTL', 'DS_GEN', 'WT_CTL']
****
 nCount_RNA
****
 nFeature_RNA
****
 Genotype
['DS', 'WT']
****
 ID
['11351', '11352', '11353', '11354', '11355', '11356', '11357', '11358', '11359', '11360', '11361', '11362']
****
 Treatment
['CTL', 'GEN']
****
 percent.mt
****
 percent.ribo
****
 log10GenesPerUMI
****
 RNA_snn_res.1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
****
 seurat_clusters
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
****
 layerI1
****
 layerV_VI_51
****
 layerV_VI_131
****
 layerV_VI_31
****
 layerV_VI_71
****
 svz1_migrating_41
****
 svz2_migrating_111
****
 svz3_prolif1
****
 rg11
****
 rg21
****
 rg31
****
 gang1
****
 int11
****
 int21
****
 stri_inh_11
****
 stri_inh_21
****
 thalamic1
****
 choroid_plexus1
****
 mic1
****
 endo1
****
 Endo1
****
 Fib1
****
 Per1
****
 SMC1
****
 Ependymal1
****
 celltype
['choroid_plexus', 'endo', 'ependymal', 'fibroblasts', 'inte

In [30]:
adata.write(dir_ds+'.'.join(fn.split('.')[:-1])+'.h5ad')